In [1]:
import os
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# Ensure CPU fallback if GPU is unavailable
tf.config.set_visible_devices([], 'GPU')


2025-04-15 07:20:12.952857: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-15 07:20:12.956118: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-15 07:20:12.966624: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744701612.983917 1093926 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744701612.989080 1093926 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744701613.002414 1093926 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
def load_parquet_data(file_path, source_name):
    """Load a parquet file and return a DataFrame"""
    if not os.path.exists(file_path):
        print(f"Error: {source_name} file not found: {file_path}")
        return None

    print(f"Loading {source_name} data from: {file_path}")
    table = pq.read_table(file_path)
    df = table.to_pandas()
    print(f"{source_name} data shape: {df.shape}")
    print(f"{source_name} columns: {df.columns.tolist()}")

    for col in df.columns:
        if df[col].dtype == 'object':
            try:
                df[col] = pd.to_numeric(df[col], errors='coerce')
                print(f"Converted column {col} from object to numeric")
            except Exception as e:
                print(f"Could not convert column {col} to numeric: {e}")

    return df

In [3]:
def normalize_timestamps(df, time_col='time_sec'):
    """Normalize timestamps to start from zero."""
    if df is None or df.empty:
        return df
    min_time = df[time_col].min()
    if pd.isna(min_time):
        print(f"Warning: No valid timestamps in {time_col}")
        return df
    df[f'{time_col}_normalized'] = df[time_col] - min_time
    print(f"Normalized timestamps: min={min_time}, range=[0, {df[time_col].max() - min_time}]")
    return df

In [4]:
def process_rss_data(rss_df, page_size_mb):
    """Process RSS data with appropriate page size conversion"""
    if rss_df is None or rss_df.empty:
        print("Warning: RSS DataFrame is empty")
        return None, []

    if 'ts_ns' in rss_df.columns:
        rss_df['time_sec'] = rss_df['ts_ns'] / 1e9
        print("Converted timestamp from ns to sec")

    rss_df = rss_df.sort_values('time_sec')
    rss_df = normalize_timestamps(rss_df)
    rss_df['page_size_mb'] = page_size_mb

    rss_cols = []
    for col in ['anon', 'file', 'swap', 'shmem']:
        if col in rss_df.columns:
            if rss_df[col].std() > 0:
                rss_cols.append(col)
                print(f"Found RSS column with variance: {col}")

    for col in rss_cols:
        new_col = f"{col}_mb"
        rss_df[new_col] = rss_df[col] * page_size_mb
        print(f"Converted {col} to MB using {page_size_mb}MB page size")

    # Ensure file_mb exists
    if 'file' not in rss_cols:
        print("Warning: 'file' column missing or has zero variance, adding file_mb with zeros")
        rss_df['file_mb'] = 0
        rss_cols.append('file')

    print(f"Processed RSS columns: {rss_df.columns.tolist()}")
    return rss_df, rss_cols

In [5]:
def process_tlb_data(tlb_df, tlb_type):
    """Process TLB miss data"""
    if tlb_df is None or tlb_df.empty:
        print(f"Warning: {tlb_type} DataFrame is empty")
        return None, []

    if 'ts_uptime_us' in tlb_df.columns:
        tlb_df['time_sec'] = tlb_df['ts_uptime_us'] / 1e6
        print(f"Converted {tlb_type} timestamp from us to sec")

    tlb_df = tlb_df.sort_values('time_sec')
    tlb_df = normalize_timestamps(tlb_df)

    tlb_cols = []
    for col in tlb_df.columns:
        if 'miss' in col.lower():
            if tlb_df[col].std() > 0:
                tlb_cols.append(col)
    print(f"Found {len(tlb_cols)} {tlb_type} columns with variance: {tlb_cols}")

    main_tlb_col = None
    for col in tlb_cols:
        if 'cumulative' in col.lower() or 'total' in col.lower():
            main_tlb_col = col
            break

    if main_tlb_col is None and tlb_cols:
        main_tlb_col = tlb_cols[0]

    if main_tlb_col:
        print(f"Selected {tlb_type} column: {main_tlb_col}")
        tlb_df[f'{tlb_type}_misses'] = tlb_df[main_tlb_col]
        tlb_df[f'{tlb_type}_misses_rate'] = tlb_df[f'{tlb_type}_misses'].diff() / tlb_df['time_sec_normalized'].diff()
        tlb_df[f'{tlb_type}_misses_rate'].fillna(0, inplace=True)
        print(f"Generated TLB columns: {[f'{tlb_type}_misses', f'{tlb_type}_misses_rate']}")
        return tlb_df, [f'{tlb_type}_misses', f'{tlb_type}_misses_rate']
    else:
        print(f"Warning: No suitable {tlb_type} column found")
        return tlb_df, []

In [6]:

def create_memory_change_dataset(kb2_data, kb4_data):
    """Create a dataset aligning memory changes with TLB measurements"""
    print("Creating memory-change aligned dataset...")
    kb2_rss = kb2_data.get('rss')
    kb2_dtlb = kb2_data.get('dtlb')
    kb2_itlb = kb2_data.get('itlb')
    kb4_rss = kb4_data.get('rss')
    kb4_dtlb = kb4_data.get('dtlb')
    kb4_itlb = kb4_data.get('itlb')

    if kb2_rss is None or kb4_rss is None:
        raise ValueError("Both 2MB and 4KB RSS data are required")

    def deduplicate_timestamps(df, time_col='time_sec_normalized'):
        if df is None:
            return None
        if df[time_col].duplicated().any():
            print(f"Found duplicate timestamps in {time_col}; removing duplicates...")
            df = df.sort_values(time_col).drop_duplicates(subset=[time_col], keep='last')
        return df

    for name, df in [('2MB RSS', kb2_rss), ('2MB DTLB', kb2_dtlb), ('2MB ITLB', kb2_itlb),
                     ('4KB RSS', kb4_rss), ('4KB DTLB', kb4_dtlb), ('4KB ITLB', kb4_itlb)]:
        if df is not None:
            df = deduplicate_timestamps(df)
            if name == '2MB RSS': kb2_rss = df
            elif name == '2MB DTLB': kb2_dtlb = df
            elif name == '2MB ITLB': kb2_itlb = df
            elif name == '4KB RSS': kb4_rss = df
            elif name == '4KB DTLB': kb4_dtlb = df
            elif name == '4KB ITLB': kb4_itlb = df

    def find_memory_changes(df, memory_cols, threshold=0.001):
        if df is None:
            return pd.DataFrame()
        change_points = []
        for col in memory_cols:
            col_mb = f"{col}_mb"
            if col_mb in df.columns:
                diff = df[col_mb].diff().abs()
                changes = df[diff > threshold].copy()
                if not changes.empty:
                    changes['memory_metric'] = col_mb
                    changes['memory_value'] = changes[col_mb]
                    change_points.append(changes[['time_sec_normalized', 'memory_metric', 'memory_value']])
        if not change_points:
            return pd.DataFrame()
        return pd.concat(change_points).sort_values('time_sec_normalized')

    kb2_memory_candidates = [col for col in ['anon', 'file'] if col in kb2_rss.columns]
    kb4_memory_candidates = [col for col in ['anon', 'file'] if col in kb4_rss.columns]

    kb2_changes = find_memory_changes(kb2_rss, kb2_memory_candidates)
    kb4_changes = find_memory_changes(kb4_rss, kb4_memory_candidates)

    print(f"Found {len(kb2_changes)} memory change points in 2MB data")
    print(f"Found {len(kb4_changes)} memory change points in 4KB data")

    def align_with_tlb(changes_df, dtlb_df, itlb_df):
        if changes_df is None or changes_df.empty:
            return changes_df
        result = changes_df.copy()
        if dtlb_df is not None:
            dtlb_df = dtlb_df.sort_values('time_sec_normalized')
            dtlb_columns = [col for col in dtlb_df.columns
                           if col in ['2MB_DTLB_misses', '2MB_DTLB_misses_rate', '4KB_DTLB_misses', '4KB_DTLB_misses_rate']]
            if dtlb_columns:
                result = pd.merge_asof(
                    result,
                    dtlb_df[['time_sec_normalized'] + dtlb_columns],
                    on='time_sec_normalized',
                    direction='backward',
                    suffixes=("", "_dtlb")
                )
                print(f"DTLB merged columns: {dtlb_columns}")
            else:
                print("Warning: No valid DTLB columns to merge")
        if itlb_df is not None:
            itlb_df = itlb_df.sort_values('time_sec_normalized')
            itlb_columns = [col for col in itlb_df.columns
                           if col in ['2MB_ITLB_misses', '2MB_ITLB_misses_rate', '4KB_ITLB_misses', '4KB_ITLB_misses_rate']]
            if itlb_columns:
                result = pd.merge_asof(
                    result,
                    itlb_df[['time_sec_normalized'] + itlb_columns],
                    on='time_sec_normalized',
                    direction='backward',
                    suffixes=("", "_itlb")
                )
                print(f"ITLB merged columns: {itlb_columns}")
            else:
                print("Warning: No valid ITLB columns to merge")
        print(f"Aligned DataFrame columns: {result.columns.tolist()}")
        return result

    kb2_df = align_with_tlb(kb2_changes, kb2_dtlb, kb2_itlb)
    kb4_df = align_with_tlb(kb4_changes, kb4_dtlb, kb4_itlb)

    return {
        'kb2_df': kb2_df,
        'kb4_df': kb4_df,
        'kb2_memory_metrics': kb2_df['memory_metric'].unique().tolist() if not kb2_df.empty else [],
        'kb4_memory_metrics': kb4_df['memory_metric'].unique().tolist() if not kb4_df.empty else []
    }


In [7]:
def prepare_cross_config_data(integrated_data, target_metric='anon_mb', window_size=10.0, step_size=2.0):
    """Prepare data for model evaluation"""
    kb2_df = integrated_data.get('kb2_df')
    kb4_df = integrated_data.get('kb4_df')
    
    if kb2_df is None or kb2_df.empty or kb4_df is None or kb4_df.empty:
        raise ValueError("Both 2MB and 4KB integrated data are required")

    print(f"kb2_df columns: {kb2_df.columns.tolist()}")
    print(f"kb4_df columns: {kb4_df.columns.tolist()}")
    print(f"kb4_df memory_metric values: {kb4_df['memory_metric'].unique().tolist()}")
    available_metrics = kb4_df['memory_metric'].unique()
    if target_metric not in available_metrics:
        print(f"Warning: Target metric {target_metric} not found. Available: {available_metrics}")
        return pd.DataFrame()  # Return empty DataFrame to prevent further errors

    kb4_targets = kb4_df[kb4_df['memory_metric'] == target_metric][['time_sec_normalized', 'memory_value']].copy()
    kb4_targets = kb4_targets.rename(columns={'memory_value': '4kb_target'})
    print(f"kb4_targets shape: {kb4_targets.shape}")
    if kb4_targets.empty:
        print("Error: No data found for target_metric in kb4_df")
        return pd.DataFrame()

    kb2_rss = kb2_df[kb2_df['memory_metric'] == target_metric][['time_sec_normalized', 'memory_value']].copy()
    kb2_rss = kb2_rss.rename(columns={'memory_value': '2mb_rss'})
    print(f"kb2_rss shape: {kb2_rss.shape}")

    kb4_targets = kb4_targets.sort_values('time_sec_normalized')
    kb2_df = kb2_df.sort_values('time_sec_normalized')
    kb2_rss = kb2_rss.sort_values('time_sec_normalized')

    min_time = min(kb4_targets['time_sec_normalized'].min(), kb2_df['time_sec_normalized'].min())
    max_time = max(kb4_targets['time_sec_normalized'].max(), kb2_df['time_sec_normalized'].max())
    window_starts = np.arange(min_time, max_time - window_size + step_size, step_size)
    print(f"Window range: [{min_time}, {max_time}], {len(window_starts)} windows")

    window_data = []
    expected_features = ['memory_value', 'file_mb', '2MB_DTLB_misses', '2MB_DTLB_misses_rate',
                        '2MB_ITLB_misses', '2MB_ITLB_misses_rate']
    kb2_features = [col for col in expected_features if col in kb2_df.columns]
    print(f"Available kb2_features: {kb2_features}")
    
    if len(kb2_features) < 6:
        print(f"Warning: Only {len(kb2_features)} features available, padding with zeros")
        for i in range(6 - len(kb2_features)):
            kb2_df[f'dummy_feature_{i}'] = 0
            kb2_features.append(f'dummy_feature_{i}')
    
    for start in window_starts:
        end = start + window_size
        kb2_window = kb2_df[(kb2_df['time_sec_normalized'] >= start) & 
                           (kb2_df['time_sec_normalized'] < end)]
        kb4_window = kb4_targets[(kb4_targets['time_sec_normalized'] >= start) & 
                                (kb4_targets['time_sec_normalized'] < end)]
        kb2_rss_window = kb2_rss[(kb2_rss['time_sec_normalized'] >= start) & 
                                (kb2_rss['time_sec_normalized'] < end)]
        
        kb4_target = kb4_window['4kb_target'].iloc[-1] if not kb4_window.empty else np.nan
        kb2_rss_value = kb2_rss_window['2mb_rss'].iloc[-1] if not kb2_rss_window.empty else np.nan
        kb2_agg = kb2_window[kb2_features].replace([np.inf, -np.inf], np.nan).mean().to_dict() if not kb2_window.empty else {col: np.nan for col in kb2_features}
        
        # Append even if kb4_target is nan to avoid empty DataFrame
        window_data.append({
            'window_start': start,
            'window_end': end,
            'time_sec': (start + end) / 2,
            '4kb_target': kb4_target,
            '2mb_rss': kb2_rss_value,
            **kb2_agg
        })
    
    cross_config_df = pd.DataFrame(window_data)
    print(f"Initial cross_config_df shape: {cross_config_df.shape}")
    print(f"cross_config_df columns: {cross_config_df.columns.tolist()}")
    
    if cross_config_df.empty:
        print("Warning: No windows generated, returning empty DataFrame")
        return cross_config_df
    
    cross_config_df[kb2_features] = cross_config_df[kb2_features].fillna(0)
    # Only dropna if we have data to avoid KeyError
    if '4kb_target' in cross_config_df.columns:
        cross_config_df = cross_config_df.dropna(subset=['4kb_target'])
        print(f"After dropna cross_config_df shape: {cross_config_df.shape}")
    else:
        print("Warning: '4kb_target' column missing in cross_config_df")
    
    return cross_config_df


In [8]:

def prepare_lstm_sequences(df, target_col, feature_cols, seq_length=5):
    """Prepare sequences for LSTM model"""
    if df.empty:
        print("Error: Input DataFrame is empty, cannot prepare sequences")
        return np.array([]), np.array([]), {}, []

    expected_features = ['memory_value', 'file_mb', '2MB_DTLB_misses', '2MB_DTLB_misses_rate',
                        '2MB_ITLB_misses', '2MB_ITLB_misses_rate']
    valid_feature_cols = []
    for col in expected_features:
        if col in df.columns and pd.api.types.is_numeric_dtype(df[col]) and df[col].std() > 0:
            valid_feature_cols.append(col)
    
    print(f"Valid feature columns before padding: {valid_feature_cols}")
    if len(valid_feature_cols) < 6:
        print(f"Warning: Only {len(valid_feature_cols)} features with variance, padding with zeros")
        for i in range(6 - len(valid_feature_cols)):
            col = f'dummy_feature_{i}'
            df[col] = 0
            valid_feature_cols.append(col)
    elif len(valid_feature_cols) > 6:
        print(f"Warning: Found {len(valid_feature_cols)} features, trimming to 6")
        valid_feature_cols = valid_feature_cols[:6]
    
    print(f"Final valid feature columns: {valid_feature_cols}")
    scalers = {}
    scaled_data = {}
    for col in valid_feature_cols + [target_col]:
        scaler = MinMaxScaler()
        data = df[col].values.reshape(-1, 1)
        scaled_data[col] = scaler.fit_transform(data)
        scalers[col] = scaler
    
    X, y = [], []
    for i in range(len(df) - seq_length):
        features_seq = [scaled_data[col][i:i+seq_length] for col in valid_feature_cols]
        X.append(np.hstack(features_seq))
        y.append(scaled_data[target_col][i+seq_length])
    
    X, y = np.array(X), np.array(y)
    X = X.reshape((X.shape[0], seq_length, len(valid_feature_cols)))
    print(f"Generated X with shape: {X.shape}")
    
    return X, y, scalers, valid_feature_cols


In [9]:
def visualize_predictions(actual_values, predicted_values, kb2_rss_values, time_steps=None, save_path='output/evaluation_prediction_comparison.png'):
    """Visualize model predictions"""
    actual = np.array(actual_values).flatten()
    predicted = np.array(predicted_values).flatten()
    kb2_rss = np.array(kb2_rss_values).flatten()

    if np.all(np.isnan(actual)) or np.all(np.isnan(predicted)):
        print(f"Warning: Invalid data for visualization at {save_path}")
        return {
            'mse': np.nan,
            'rmse': np.nan,
            'mae': np.nan,
            'mape': np.nan,
            'r2': np.nan,
            'correlation': np.nan
        }

    x_values = time_steps if time_steps is not None else np.arange(len(actual))
    x_label = 'Time (seconds)' if time_steps is not None else 'Sample Index'

    error = actual - predicted
    mse = np.nanmean(np.square(error))
    rmse = np.sqrt(mse) if not np.isnan(mse) else np.nan
    mae = np.nanmean(np.abs(error))
    mape = np.nanmean(np.abs(error / (actual + 1e-10))) * 100
    r2 = 1 - (np.nansum(np.square(error)) / np.nansum(np.square(actual - np.nanmean(actual)))) if not np.all(np.isnan(actual)) else np.nan
    corr = np.corrcoef(actual, predicted)[0, 1] if not (np.all(np.isnan(actual)) or np.all(np.isnan(predicted))) else np.nan

    plt.figure(figsize=(15, 10))
    plt.plot(x_values, actual, 'b-', label='Actual 4KB Memory')
    plt.plot(x_values, predicted, 'r--', label='Predicted 4KB Memory')
    plt.plot(x_values, kb2_rss, 'g-.', label='Actual 2MB Memory')
    plt.title('Memory Usage Comparison: 4KB (Actual vs Predicted) and 2MB', fontsize=14)
    plt.xlabel(x_label, fontsize=12)
    plt.ylabel('Memory Usage (MB)', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.legend(fontsize=10)
    
    metrics_text = f"MSE: {mse:.4f}\nRMSE: {rmse:.4f}\nMAE: {mae:.4f}\nMAPE: {mape:.2f}%\nR²: {r2:.4f}"
    plt.annotate(metrics_text, xy=(0.02, 0.85), xycoords='axes fraction',
                 bbox=dict(boxstyle="round,pad=0.5", fc="white", ec="gray", alpha=0.8),
                 fontsize=10)

    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Visualization saved to {save_path}")

    return {
        'mse': mse,
        'rmse': rmse,
        'mae': mae,
        'mape': mape,
        'r2': r2,
        'correlation': corr
    }


In [11]:
def main(kb2_rss_path, kb4_rss_path, kb2_dtlb_path, kb2_itlb_path, kb4_dtlb_path, kb4_itlb_path, model_path='output/cross_page_prediction_model.keras'):
    """Evaluate the cross-page prediction model with RSS and TLB data"""
    try:
        os.makedirs('output', exist_ok=True)
        print("="*50)
        print("EVALUATING CROSS-PAGE MEMORY PREDICTION MODEL")
        print("="*50)

        print("\n=== Loading Model ===")
        if not os.path.exists(model_path):
            raise FileNotFoundError(f"Model file not found at {model_path}")
        model = tf.keras.models.load_model(model_path)
        print(f"Model loaded from {model_path}")

        print("\n=== Loading Data ===")
        kb2_data = {
            'rss': load_parquet_data(kb2_rss_path, "2MB RSS"),
            'dtlb': load_parquet_data(kb2_dtlb_path, "2MB DTLB"),
            'itlb': load_parquet_data(kb2_itlb_path, "2MB ITLB")
        }
        kb4_data = {
            'rss': load_parquet_data(kb4_rss_path, "4KB RSS"),
            'dtlb': load_parquet_data(kb4_dtlb_path, "4KB DTLB"),
            'itlb': load_parquet_data(kb4_itlb_path, "4KB ITLB")
        }

        print("\n=== Processing Data ===")
        if kb2_data['rss'] is not None:
            kb2_data['rss'], kb2_rss_cols = process_rss_data(kb2_data['rss'], page_size_mb=0.004)
        else:
            raise ValueError("Failed to load 2MB RSS data")
        if kb4_data['rss'] is not None:
            kb4_data['rss'], kb4_rss_cols = process_rss_data(kb4_data['rss'], page_size_mb=0.004)
        else:
            raise ValueError("Failed to load 4KB RSS data")
        
        if kb2_data['dtlb'] is not None:
            kb2_data['dtlb'], kb2_dtlb_cols = process_tlb_data(kb2_data['dtlb'], "2MB_DTLB")
        else:
            print("Warning: 2MB DTLB data missing")
            kb2_data['dtlb'] = None
        if kb2_data['itlb'] is not None:
            kb2_data['itlb'], kb2_itlb_cols = process_tlb_data(kb2_data['itlb'], "2MB_ITLB")
        else:
            print("Warning: 2MB ITLB data missing")
            kb2_data['itlb'] = None
        
        if kb4_data['dtlb'] is not None:
            kb4_data['dtlb'], kb4_dtlb_cols = process_tlb_data(kb4_data['dtlb'], "4KB_DTLB")
        else:
            print("Warning: 4KB DTLB data missing")
            kb4_data['dtlb'] = None
        if kb4_data['itlb'] is not None:
            kb4_data['itlb'], kb4_itlb_cols = process_tlb_data(kb4_data['itlb'], "4KB_ITLB")
        else:
            print("Warning: 4KB ITLB data missing")
            kb4_data['itlb'] = None

        print("\n=== Creating Integrated Dataset ===")
        integrated_data = create_memory_change_dataset(kb2_data, kb4_data)

        print("\n=== Preparing Data ===")
        cross_config_df = prepare_cross_config_data(integrated_data, target_metric='anon_mb')
        if cross_config_df.empty:
            print("Error: Failed to generate evaluation dataset")
            return None, None
        cross_config_df.to_csv('output/evaluation_dataset.csv', index=False)
        print("Evaluation dataset saved to output/evaluation_dataset.csv")

        print("\n=== Preparing LSTM Sequences ===")
        target_column = '4kb_target'
        feature_columns = ['memory_value', 'file_mb', '2MB_DTLB_misses', '2MB_DTLB_misses_rate',
                          '2MB_ITLB_misses', '2MB_ITLB_misses_rate']
        X, y, scalers, valid_features = prepare_lstm_sequences(
            cross_config_df, target_column, feature_columns, seq_length=5
        )

        if len(X) < 1:
            print("Not enough data for evaluation")
            return None, None

        print("\n=== Making Predictions ===")
        y_pred = model.predict(X)

        y_actual = scalers[target_column].inverse_transform(y.reshape(-1, 1)).flatten()
        y_pred = scalers[target_column].inverse_transform(y_pred).flatten()

        kb2_rss_values = cross_config_df['2mb_rss'].values[5:5+len(y_actual)]
        time_values = cross_config_df['time_sec'].values[5:5+len(y_actual)]

        print("\n=== Evaluating Performance ===")
        metrics = visualize_predictions(
            actual_values=y_actual,
            predicted_values=y_pred,
            kb2_rss_values=kb2_rss_values,
            time_steps=time_values,
            save_path='output/evaluation_prediction_comparison.png'
        )

        with open('output/evaluation_metrics.txt', 'w') as f:
            for k, v in metrics.items():
                f.write(f"{k}: {v}\n")
        print("Evaluation metrics saved to output/evaluation_metrics.txt")

        print("\n=== Evaluation Complete ===")
        print(f"RMSE: {metrics['rmse']:.4f}, MAPE: {metrics['mape']:.2f}%")
        return model, metrics

    except Exception as e:
        print(f"Error during evaluation: {e}")
        import traceback
        traceback.print_exc()
        return None, None

if __name__ == "__main__":
    kb2_rss_path = "data/curated/mm_rss_stat/ee6f283a-1848-46a8-a741-cf7cda9368e3.redis.parquet"
    kb4_rss_path = "data/curated/mm_rss_stat/386c4cbd-3fa6-408a-84ef-4f31422eef80.redis.parquet"
    kb2_dtlb_path = "data/curated/dtlb_misses/ee6f283a-1848-46a8-a741-cf7cda9368e3.redis.parquet"
    kb2_itlb_path = "data/curated/itlb_misses/ee6f283a-1848-46a8-a741-cf7cda9368e3.redis.parquet"
    kb4_dtlb_path = "data/curated/dtlb_misses/386c4cbd-3fa6-408a-84ef-4f31422eef80.redis.parquet"
    kb4_itlb_path = "data/curated/itlb_misses/386c4cbd-3fa6-408a-84ef-4f31422eef80.redis.parquet"
    model, metrics = main(kb2_rss_path, kb4_rss_path, kb2_dtlb_path, kb2_itlb_path, kb4_dtlb_path, kb4_itlb_path)

EVALUATING CROSS-PAGE MEMORY PREDICTION MODEL

=== Loading Model ===
Model loaded from output/cross_page_prediction_model.keras

=== Loading Data ===
Error: 2MB RSS file not found: data/curated/mm_rss_stat/ee6f283a-1848-46a8-a741-cf7cda9368e3.redis.parquet
Error: 2MB DTLB file not found: data/curated/dtlb_misses/ee6f283a-1848-46a8-a741-cf7cda9368e3.redis.parquet
Error: 2MB ITLB file not found: data/curated/itlb_misses/ee6f283a-1848-46a8-a741-cf7cda9368e3.redis.parquet
Error: 4KB RSS file not found: data/curated/mm_rss_stat/386c4cbd-3fa6-408a-84ef-4f31422eef80.redis.parquet
Error: 4KB DTLB file not found: data/curated/dtlb_misses/386c4cbd-3fa6-408a-84ef-4f31422eef80.redis.parquet
Error: 4KB ITLB file not found: data/curated/itlb_misses/386c4cbd-3fa6-408a-84ef-4f31422eef80.redis.parquet

=== Processing Data ===
Error during evaluation: Failed to load 2MB RSS data


Traceback (most recent call last):
  File "/tmp/ipykernel_1093926/2050444355.py", line 31, in main
    raise ValueError("Failed to load 2MB RSS data")
ValueError: Failed to load 2MB RSS data
